In [ ]:
import pandas as pd
import numpy as np
import cv2 as cv
import cv2
import glob
import imutils
from imutils import paths
import os
import os.path

In [ ]:
f = open("labels.txt",'r')
f1 = f.readlines()

In [ ]:
import cv2

SOLVED_CAPTCHA_FOLDER = "drive/MyDrive/train"
OUTPUT_FOLDER = "extracted_letters"


# Get the path of all the solved captcha images
solved_captchas = glob.glob(os.path.join(SOLVED_CAPTCHA_FOLDER, "*"))
counts = {}
c = 0
# loop over the image paths
for (i, captcha) in enumerate(solved_captchas):
    print("processing image {}/{}".format(i + 1, len(solved_captchas)))

    
    # grab the base filename as the text
    filename = os.path.basename(captcha)
    captcha_text = os.path.splitext(filename)[0]

    img = cv.imread(captcha,-1)
    hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    lower_red = np.array([0,0,150])
    upper_red = np.array([255,255,255])
    # Load the image and convert it to grayscale
#     image = cv2.imread(captcha)
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    mask = cv.inRange(hsv, lower_red, upper_red)
    imCopy = mask.copy()
    imCopy=~imCopy
#     # Add some extra padding around the image
#     gray = cv2.copyMakeBorder(gray, 8, 8, 8, 8, cv2.BORDER_REPLICATE)

#     # applying threshold
#     thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV, cv2.THRESH_OTSU)[1]
    contours,hierarchy= cv.findContours(imCopy, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)[-2:]
    letter_image_regions = []
#      # finding the contours
#     contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     # creating empty list for holding the coordinates of the letters
#     letter_image_regions = []


    # Now we will loop through each of the contours and extract the letter
    for contour in contours:
        # Get the rectangle that contains the contour
        (x, y, w, h) = cv2.boundingRect(contour)
        
        # checking if any counter is too wide
        # if countour is too wide then there could be two letters joined together or are very close to each other
        letter_image_regions.append((x, y, w, h))
            

    
    # Sort the detected letter images based on the x coordinate to make sure
    # we get them from left-to-right so that we match the right image with the right letter
    letter_image_regions = sorted(letter_image_regions, key=lambda x: x[2]*x[3], reverse=True)  
    letter_image_regions = letter_image_regions[0:3]
    letter_image_regions = sorted(letter_image_regions, key=lambda x: x[0])
    
    # Save each letter as a single image
    for letter_bounding_box in letter_image_regions:
        # Grab the coordinates of the letter in the image
        x, y, w, h = letter_bounding_box

        # Extract the letter from the original image with a 2-pixel margin around the edge
        letter_image = mask[y - 2:y + h + 2, x - 2:x + w + 2]

        letter_text = f1[int(captcha_text)].strip()
        letter_text = letter_text.split(',')
        l1 = letter_text[c]
        # Get the folder to save the image in
        save_path = os.path.join(OUTPUT_FOLDER, l1)

        # creating different output folder for storing different letters
        if not os.path.exists(save_path):
            os.makedirs(save_path)

        # write the letter image to a file
        count = counts.get(l1, 1)
        p = os.path.join(save_path, "{}.png".format(str(count)))
        cv2.imwrite(p, letter_image)
        #cv2.imshow('image',letter_image)

        # increment the count
        counts[l1] = count + 1
        c = (c+1)%3

processing image 1/2000
processing image 2/2000
processing image 3/2000
processing image 4/2000
processing image 5/2000
processing image 6/2000
processing image 7/2000
processing image 8/2000
processing image 9/2000
processing image 10/2000
processing image 11/2000
processing image 12/2000
processing image 13/2000
processing image 14/2000
processing image 15/2000
processing image 16/2000
processing image 17/2000
processing image 18/2000
processing image 19/2000
processing image 20/2000
processing image 21/2000
processing image 22/2000
processing image 23/2000
processing image 24/2000
processing image 25/2000
processing image 26/2000
processing image 27/2000
processing image 28/2000
processing image 29/2000
processing image 30/2000
processing image 31/2000
processing image 32/2000
processing image 33/2000
processing image 34/2000
processing image 35/2000
processing image 36/2000
processing image 37/2000
processing image 38/2000
processing image 39/2000
processing image 40/2000
processin

In [ ]:
!zip -r '/content/NewFileName.zip' '/content/extracted_letters'

Streaming output truncated to the last 5000 lines.
  adding: content/extracted_letters/XI/ (stored 0%)
  adding: content/extracted_letters/XI/104.png (stored 0%)
  adding: content/extracted_letters/XI/24.png (deflated 2%)
  adding: content/extracted_letters/XI/76.png (stored 0%)
  adding: content/extracted_letters/XI/92.png (deflated 5%)
  adding: content/extracted_letters/XI/234.png (stored 0%)
  adding: content/extracted_letters/XI/18.png (stored 0%)
  adding: content/extracted_letters/XI/40.png (deflated 0%)
  adding: content/extracted_letters/XI/229.png (deflated 1%)
  adding: content/extracted_letters/XI/121.png (deflated 1%)
  adding: content/extracted_letters/XI/150.png (deflated 35%)
  adding: content/extracted_letters/XI/51.png (stored 0%)
  adding: content/extracted_letters/XI/236.png (deflated 0%)
  adding: content/extracted_letters/XI/66.png (deflated 4%)
  adding: content/extracted_letters/XI/168.png (stored 0%)
  adding: content/extracted_letters/XI/80.png (deflated 35%)


In [ ]:
letter_folder = 'drive/MyDrive/extracted_letters'

#creating empty lists for storing image data and labels
data = []
labels = []
for image in paths.list_images(letter_folder):
    img = cv2.imread(image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (30,30))
    
    # adding a 3rd dimension to the image
    img = np.expand_dims(img, axis = 2)
    
    #grabing the name of the letter based on the folder it is present in
    label = image.split(os.path.sep)[-2]
    
    # appending to the empty lists
    data.append(img)
    labels.append(label)

#converting data and labels to np array
data = np.array(data, dtype = "float")
labels = np.array(labels)

array(['PI', 'PI', 'PI', ..., 'BETA', 'BETA', 'BETA'], dtype='<U7')

In [ ]:
#scaling the values of  data between 0 and 1
data = data/255.0

# Split the training data into separate train and test sets
from sklearn.model_selection import train_test_split
(train_x, val_x, train_y, val_y) = train_test_split(data, labels, test_size=0.2, random_state=0)

#one hot encoding
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer().fit(train_y)
train_y = lb.transform(train_y)
val_y = lb.transform(val_y)

In [ ]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping

#building model
model = Sequential()
model.add(Conv2D(20, (5, 5), padding="same", input_shape=(30, 30, 1), activation="relu"))
model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5)))
model.add(Conv2D(50, (5, 5), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5)))


model.add(Flatten())
model.add(Dense(80, activation="relu"))

model.add(Dropout(0.3))
model.add(Dense(24, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# using early stoping for avoiding overfitting
estop = EarlyStopping(patience=10, mode='min', min_delta=0.001, monitor='val_loss')

model.fit(train_x, train_y, validation_data=(val_x, val_y), batch_size=32, epochs=30, verbose=1, callbacks = [estop])
model.save('my_model_1.h5')

Epoch 1/30
150/150 [==============================] - 6s 36ms/step - loss: 2.5937 - accuracy: 0.2748 - val_loss: 1.2253 - val_accuracy: 0.8408
Epoch 2/30
150/150 [==============================] - 5s 33ms/step - loss: 0.6584 - accuracy: 0.8350 - val_loss: 0.1117 - val_accuracy: 1.0000
Epoch 3/30
150/150 [==============================] - 5s 34ms/step - loss: 0.1634 - accuracy: 0.9700 - val_loss: 0.0229 - val_accuracy: 1.0000
Epoch 4/30
150/150 [==============================] - 5s 33ms/step - loss: 0.0691 - accuracy: 0.9906 - val_loss: 0.0071 - val_accuracy: 1.0000
Epoch 5/30
150/150 [==============================] - 5s 33ms/step - loss: 0.0450 - accuracy: 0.9942 - val_loss: 0.0035 - val_accuracy: 1.0000
Epoch 6/30
150/150 [==============================] - 5s 34ms/step - loss: 0.0267 - accuracy: 0.9975 - val_loss: 0.0013 - val_accuracy: 1.0000
Epoch 7/30
150/150 [==============================] - 5s 32ms/step - loss: 0.0220 - accuracy: 0.9973 - val_loss: 0.0014 - val_accuracy: 1.0000

In [ ]:
import cv2 as cv
import numpy as np
from itertools import groupby 
from keras.models import load_model
# from sklearn.preprocessing import LabelBinarizer

# DO NOT CHANGE THE NAME OF THIS METHOD OR ITS INPUT OUTPUT BEHAVIOR

# INPUT CONVENTION
# filenames: a list of strings containing filenames of images

# OUTPUT CONVENTION
# The method must return a numpy array (not numpy matrix or scipy matrix) and a list of strings.
# Make sure that the length of the array and the list is the same as the number of filenames that 
# were given. The evaluation code may give unexpected results if this convention is not followed.

label = np.array(["ALPHA","BETA","CHI","DELTA","EPSILON","ETA","GAMMA","IOTA","KAPPA","LAMDA","MU","NU","OMEGA","OMICRON","PHI","PI","PSI","RHO","SIGMA","TAU","THETA","UPSLION","XI","ZETA"])

def inverse_t(pred):
  # print(pred[0])
  for i in range(len(pred[0])):
    if pred[0][i] == 1:
      return label[i]

def decaptcha( filenames ):
    model = load_model('my_model_1.h5')
    codes=[]
    for filename in filenames:
        image = cv.imread(filename)
       
        hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
        lower_gray = np.array([0,0,150])
        upper_gray = np.array([255,255,255])
       
        mask = cv.inRange(hsv, lower_gray, upper_gray)
        imCopy = mask.copy()
        imCopy=~imCopy
       
        contours,hierarchy= cv.findContours(imCopy, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

        letter_image_regions = []

        # Now we can loop through each of the contours and extract the letter

        for contour in contours:
            # Get the rectangle that contains the contour
            (x, y, w, h) = cv.boundingRect(contour)
            letter_image_regions.append((x, y, w, h))
                    

        # Sort the detected letter images based on the x coordinate to make sure
        # we get them from left-to-right so that we match the right image with the right letter
        letter_image_regions = sorted(letter_image_regions, key=lambda x: x[2]*x[3], reverse=True)  
        letter_image_regions = letter_image_regions[0:3]
        letter_image_regions = sorted(letter_image_regions, key=lambda x: x[0])

        # Create an output image and a list to hold our predicted letters
        output = cv.merge([mask] * 3)
        predictions = []
            
        # Creating an empty list for storing predicted letters
        predictions = []
        print(len(letter_image_regions))
        # Save out each letter as a single image
        for letter_bounding_box in letter_image_regions:
            # Grab the coordinates of the letter in the image
            x, y, w, h = letter_bounding_box

            # Extract the letter from the original image with a 2-pixel margin around the edge
            letter_image = mask[y - 2:y + h + 2, x - 2:x + w + 2]

            letter_image = cv.resize(letter_image, (30,30))
                
            # Turn the single image into a 4d list of images
            letter_image = np.expand_dims(letter_image, axis=2)
            letter_image = np.expand_dims(letter_image, axis=0)

            # making prediction
            pred = model.predict(letter_image)
                
            # Convert the one-hot-encoded prediction back to a normal letter
            # print(pred)
            letter = inverse_t(pred)
            predictions.append(letter)
        captcha_text = ",".join(predictions)
        codes.append(captcha_text)
    return (codes)



In [ ]:
import time as tm
import numpy as np

# # Perform longest common subsequence search on the (truncated) code and gold code
# def lcs( str1, str2, p, q ):
#     if p == 0 or q == 0:
#         return 0
#     elif str1[p-1] == str2[q-1]:
#         return 1 + lcs( str1, str2, p - 1, q - 1 )
#     else:
#         return max( lcs( str1, str2, p - 1, q ), lcs( str1, str2, p, q - 1 ) )

# def getLCS( code, goldCode ):
#     return lcs( code, goldCode, min( len( code ), 4 ), len( goldCode ) )

# If there are 100 test points and the prediction code returns 110 predictions
# then we only consider the first 100 and discard the last 10 predictions. On
# the other hand, if the code returns only 90 predictions, then we assume that
# these were predictions on the first 90 test points and evaluate accordingly

# def getCodeLengthMatch( numChars, goldNumChars ):
#     minLen = min( len( codes ), len( goldCodes ) )
#     return np.count_nonzero( numChars[0:minLen] == goldNumChars[0:minLen] )

# def getCodeMatchScore( codes, goldCodes ):
#     totScore = 0
#     for i in range( min( len( codes ), len( goldCodes ) ) ):
#         totScore += getLCS( codes[i], goldCodes[i] ) / len( goldCodes[i] )
#     return totScore
c = 0
numTest = 2000
filepaths = [ "drive/MyDrive/train/%d.png" % i for i in range(1600, numTest ) ]
# file = open( "drive/Mydrive/labels.txt", "r" )
# goldCodes = file.read().splitlines()
# file.close()
# goldNumChars = np.array( [ len( goldCodes[i] ) for i in range( len( goldCodes ) ) ] )

# Get recommendations from predict.py and time the thing
# tic = tm.perf_counter()
(codes) = decaptcha( filepaths )
c = 0

# toc = tm.perf_counter()

# print( "Total time taken is %.6f seconds " % (toc - tic) )
# print( "Fraction of code lengths that match is %.6f" % (getCodeLengthMatch( numChars, goldNumChars ) / numTest)  )
# print( "Code match score is %.6f" % (getCodeMatchScore( codes, goldCodes ) / numTest) )

3
1/1 [==============================] - 0s 18ms/step
3
1/1 [==============================] - 0s 15ms/step
3
1/1 [==============================] - 0s 16ms/step
3
1/1 [==============================] - 0s 12ms/step
3
1/1 [==============================] - 0s 12ms/step
3
1/1 [==============================] - 0s 14ms/step
3
1/1 [==============================] - 0s 14ms/step
3
1/1 [==============================] - 0s 13ms/step
3
1/1 [==============================] - 0s 13ms/step
3
1/1 [==============================] - 0s 13ms/step
3
1/1 [==============================] - 0s 12ms/step
3
1/1 [==============================] - 0s 15ms/step
3
1/1 [==============================] - 0s 13ms/step
3
1/1 [==============================] - 0s 13ms/step
3
1/1 [==============================] - 0s 14ms/step
3
1/1 [==============================] - 0s 13ms/step
3
1/1 [==============================] - 0s 14ms/step
3
1/1 [==============================] - 0s 16ms/step
3
1/1 [=====================

In [ ]:
print(codes)
c= 1600
for code in codes:
  print(code, c)
  c = c+1

['ALPHA,EPSILON,RHO', 'SIGMA,ALPHA,ZETA', 'OMEGA,NU,OMICRON', 'OMEGA,KAPPA,MU', 'UPSLION,OMICRON,THETA', 'IOTA,OMEGA,UPSLION', 'PI,SIGMA,TAU', 'BETA,PSI,ETA', 'THETA,ZETA,OMEGA', 'XI,DELTA,RHO', 'KAPPA,THETA,IOTA', 'DELTA,ALPHA,ZETA', 'UPSLION,XI,KAPPA', 'ALPHA,LAMDA,TAU', 'MU,EPSILON,ZETA', 'BETA,KAPPA,PHI', 'CHI,OMEGA,BETA', 'ALPHA,OMEGA,ALPHA', 'XI,ALPHA,MU', 'LAMDA,PSI,EPSILON', 'SIGMA,NU,ETA', 'OMICRON,UPSLION,KAPPA', 'ZETA,XI,PI', 'RHO,OMEGA,PHI', 'PHI,BETA,EPSILON', 'IOTA,ETA,RHO', 'KAPPA,PSI,XI', 'DELTA,ETA,BETA', 'THETA,LAMDA,ETA', 'NU,IOTA,BETA', 'CHI,DELTA,SIGMA', 'XI,IOTA,BETA', 'DELTA,OMICRON,TAU', 'THETA,ALPHA,OMICRON', 'OMICRON,OMICRON,TAU', 'OMICRON,PI,ALPHA', 'IOTA,BETA,OMEGA', 'NU,MU,TAU', 'PI,XI,NU', 'DELTA,RHO,XI', 'GAMMA,ETA,PSI', 'ETA,OMICRON,BETA', 'PI,MU,ALPHA', 'PSI,ZETA,EPSILON', 'DELTA,GAMMA,IOTA', 'SIGMA,BETA,IOTA', 'LAMDA,SIGMA,LAMDA', 'THETA,PHI,PI', 'IOTA,THETA,THETA', 'GAMMA,PHI,IOTA', 'PSI,NU,PHI', 'THETA,OMEGA,EPSILON', 'RHO,PHI,BETA', 'ZETA,CHI,SIGMA'

In [ ]:
import time as tm
import numpy as np

# # Perform longest common subsequence search on the (truncated) code and gold code
# def lcs( str1, str2, p, q ):
#     if p == 0 or q == 0:
#         return 0
#     elif str1[p-1] == str2[q-1]:
#         return 1 + lcs( str1, str2, p - 1, q - 1 )
#     else:
#         return max( lcs( str1, str2, p - 1, q ), lcs( str1, str2, p, q - 1 ) )

# def getLCS( code, goldCode ):
#     return lcs( code, goldCode, min( len( code ), 4 ), len( goldCode ) )

# If there are 100 test points and the prediction code returns 110 predictions
# then we only consider the first 100 and discard the last 10 predictions. On
# the other hand, if the code returns only 90 predictions, then we assume that
# these were predictions on the first 90 test points and evaluate accordingly

# def getCodeLengthMatch( numChars, goldNumChars ):
#     minLen = min( len( codes ), len( goldCodes ) )
#     return np.count_nonzero( numChars[0:minLen] == goldNumChars[0:minLen] )

# def getCodeMatchScore( codes, goldCodes ):
#     totScore = 0
#     for i in range( min( len( codes ), len( goldCodes ) ) ):
#         totScore += getLCS( codes[i], goldCodes[i] ) / len( goldCodes[i] )
#     return totScore
c = 0
numTest = 2000
filepaths = [ "0.png", "1.png"]
# file = open( "drive/Mydrive/labels.txt", "r" )
# goldCodes = file.read().splitlines()
# file.close()
# goldNumChars = np.array( [ len( goldCodes[i] ) for i in range( len( goldCodes ) ) ] )

# Get recommendations from predict.py and time the thing
# tic = tm.perf_counter()
(codes1) = decaptcha( filepaths )
c = 0

# toc = tm.perf_counter()

# print( "Total time taken is %.6f seconds " % (toc - tic) )
# print( "Fraction of code lengths that match is %.6f" % (getCodeLengthMatch( numChars, goldNumChars ) / numTest)  )
# print( "Code match score is %.6f" % (getCodeMatchScore( codes, goldCodes ) / numTest) )

3
1/1 [==============================] - 0s 12ms/step
3
1/1 [==============================] - 0s 16ms/step


In [ ]:
c= 1600
for code in codes1:
  print(code, c)
  c = c+1

RHO,ALPHA,UPSLION 1600
BETA,IOTA,BETA 1601
